# KYEWORD EXTRACTION

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
print(os.getenv("JAVA_HOME"))
# Create SparkSession
spark = SparkSession.builder\
    .config("spark.driver.memory", "16G")\
    .config("spark.driver.maxResultSize", "0") \
    .config("spark.kryoserializer.buffer.max", "2000M")\
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:4.0.0,org.neo4j:neo4j-connector-apache-spark_2.12:4.1.2_for_spark_3")\
    .getOrCreate()

/usr/lib/jvm/java-11-openjdk-amd64


22/06/21 19:21:17 WARN Utils: Your hostname, pop-os resolves to a loopback address: 127.0.1.1; using 192.168.0.178 instead (on interface enp4s0)
22/06/21 19:21:17 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/vergenter/Venvs/sparkNLPVenv/lib/python3.10/site-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/vergenter/.ivy2/cache
The jars for the packages stored in: /home/vergenter/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
org.neo4j#neo4j-connector-apache-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-00d68bf0-7081-454a-a5aa-e6d7455d4776;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;4.0.0 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.828 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.code.findbugs#annotations;3.0.1 in central
	found net.jcip#jcip-annotations;1.0 in central
	found com.google.code.findbugs#jsr305;3.0.1 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	fo

In [7]:
from pyspark import StorageLevel
articles_abstracts = spark.read.format("org.neo4j.spark.DataSource")\
  .option("url", "bolt://192.168.0.178:7687")\
  .option("authentication.basic.username", os.environ["NEO4J_LOGIN"])\
  .option("authentication.basic.password", os.environ["NEO4J_PASSWORD"]).option("query", "MATCH (n:Article) where n.language =\"en\" WITH n RETURN n.id as id,n.title +'. '+ n.abstract as text")\
  .option("partitions", "4")\
  .load()
articles_abstracts.persist(StorageLevel.DISK_ONLY)

DataFrame[id: bigint, text: string]

In [11]:
from sparknlp.base import DocumentAssembler,Pipeline,LightPipeline
from sparknlp.annotator import StopWordsCleaner,SentenceDetector,Tokenizer,YakeKeywordExtraction,BertForTokenClassification,NerConverter,LemmatizerModel,Normalizer,Word2VecModel
# Transforms the raw text into a document readable by the later stages of the
# pipeline
document_assembler = DocumentAssembler() \
    .setInputCol('text') \
    .setOutputCol('document')

# Separates the document into sentences
sentence_detector = SentenceDetector() \
    .setInputCols(['document']) \
    .setOutputCol('sentences')# \
    #.setDetectLists(True)

# Separates sentences into individial tokens (words)
tokenizer = Tokenizer() \
    .setInputCols(['sentences']) \
    .setOutputCol('tokens') \
    .setContextChars(['(', ')', '?', '!', '.', ','])

# The keyphrase extraction model. Change MinNGrams and MaxNGrams to set the
# minimum and maximum length of possible keyphrases, and change NKeywords to
# set the amount of potential keyphrases identified per document.
keywords = YakeKeywordExtraction() \
    .setInputCols('tokens') \
    .setOutputCol('keywords') \
    .setMinNGrams(2) \
    .setMaxNGrams(5) \
    .setNKeywords(10) \
    .setStopWords(StopWordsCleaner().getStopWords())


# Assemble all of these stages into a pipeline, then fit the pipeline on an
# empty data frame so it can be used to transform new inputs.
pipeline_keywords = Pipeline(stages=[
    document_assembler, 
    sentence_detector,
    tokenizer,
    keywords
])
empty_df = spark.createDataFrame([[""]]).toDF('text')
keywords_model = pipeline_keywords.fit(empty_df)

In [12]:
with_keywords = keywords_model.transform(articles_abstracts).select(F.col("id"),F.arrays_zip(F.expr("transform(keywords.metadata,i -> i['score'])"),F.col("keywords.result")).alias("keywords"))
with_keywords.cache()
# 15m 10.5s

DataFrame[id: bigint, keywords: array<struct<0:string,1:string>>]

In [13]:
with_keywords.head(1)

22/06/21 12:27:27 ERROR Neo4jDataWriter: Cannot commit the transaction because the following exception
org.neo4j.driver.exceptions.TransientException: ForsetiClient[transactionId=280, clientId=4] can't acquire ExclusiveLock{owner=ForsetiClient[transactionId=283, clientId=5]} on NODE_RELATIONSHIP_GROUP_DELETE(606840), because holders of that lock are waiting for ForsetiClient[transactionId=280, clientId=4].
 Wait list:ExclusiveLock[
Client[283] waits for [ForsetiClient[transactionId=280, clientId=4]]]
	at org.neo4j.driver.internal.util.Futures.blockingGet(Futures.java:144)
	at org.neo4j.driver.internal.InternalTransaction.run(InternalTransaction.java:60)
	at org.neo4j.driver.internal.AbstractQueryRunner.run(AbstractQueryRunner.java:37)
	at org.neo4j.spark.writer.BaseDataWriter.writeBatch(BaseDataWriter.scala:64)
	at org.neo4j.spark.writer.BaseDataWriter.writeBatch(BaseDataWriter.scala:93)
	at org.neo4j.spark.writer.BaseDataWriter.writeBatch(BaseDataWriter.scala:93)
	at org.neo4j.spark.w

[Row(id=0, keywords=[Row(0='0.29808715592209734', 1='evasion attacks'), Row(0='0.3319272893390838', 1='machine learning'), Row(0='0.41272376995293325', 1='test time'), Row(0='0.3319272893390838', 1='machine learning'), Row(0='0.9848120123014036', 1='learning depends'), Row(0='1.0331193081207946', 1='attack scenario'), Row(0='0.41272376995293325', 1='test time'), Row(0='1.0331193081207946', 1='manipulating attack'), Row(0='0.4086077265811094', 1='attack samples'), Row(0='0.29808715592209734', 1='evasion attacks'), Row(0='1.1075239343161545', 1='simulate attack'), Row(0='1.1075239343161545', 1='attack scenarios'), Row(0='1.1075239343161545', 1='manipulate attack'), Row(0='0.4086077265811094', 1='attack samples'), Row(0='0.29808715592209734', 1='evasion attacks')])]

In [20]:
query1 ="""UNWIND event.keywords as keyword
MERGE (k:Keyword{word:keyword.`1`})"""

query2 ="""match (a:Article{id:event.id})
UNWIND event.keywords as keyword
match (k:Keyword{word:keyword.`1`})
with a,k,toInteger(keyword.`0`) as score
MERGE (a)-[rel:ABOUT]->(k)
ON CREATE SET rel.score = score
ON MATCH SET rel.score = CASE WHEN rel.score > score THEN rel.score ELSE score END"""



with_keywords.write.format("org.neo4j.spark.DataSource")\
.option("url", "bolt://192.168.0.178:7687")\
.option("authentication.basic.username", os.environ["NEO4J_LOGIN"])\
.option("authentication.basic.password", os.environ["NEO4J_PASSWORD"])\
.option("transaction.retries", 5)\
.option("transaction.retry.timeout", 5)\
.mode("Overwrite")\
.option("query", query1)\
.save()

with_keywords.write.format("org.neo4j.spark.DataSource")\
.option("url", "bolt://192.168.0.178:7687")\
.option("authentication.basic.username", os.environ["NEO4J_LOGIN"])\
.option("authentication.basic.password", os.environ["NEO4J_PASSWORD"])\
.option("transaction.retries", 5)\
.option("transaction.retry.timeout", 5)\
.mode("Append")\
.option("query", query2)\
.save()


In [119]:
with_keywords.describe

<bound method DataFrame.describe of DataFrame[id: bigint, keywords: array<struct<0:string,1:string>>]>

In [ ]:
# query = """match (a:Article{id:event.id})
    # set a.language=event.lang"""

    # articles_abstracts_with_lang.write.format("org.neo4j.spark.DataSource")\
    # .option("url", "bolt://192.168.0.178:7687")\
    # .option("authentication.basic.username", os.environ["NEO4J_LOGIN"])\
    # .option("authentication.basic.password", os.environ["NEO4J_PASSWORD"])\
    # .mode("append")\
    # .option("query", query)\
    # .save()


In [51]:
from sparknlp.annotator import Doc2VecModel
document = DocumentAssembler()\
  .setInputCol("text")\
  .setOutputCol("document")

token = Tokenizer()\
  .setInputCols("document")\
  .setOutputCol("token")

norm = Normalizer()\
  .setInputCols(["token"])\
  .setOutputCol("normalized")\
  .setLowercase(True)

stops = StopWordsCleaner.pretrained()\
  .setInputCols("normalized")\
  .setOutputCol("cleanedToken")
  
doc2Vec = Doc2VecModel.pretrained("doc2vec_gigaword_wiki_300", "en")\
  .setInputCols("cleanedToken")\
  .setOutputCol("sentence_embeddings")
  
pipeline_doc2vec = Pipeline(stages=[
  document,
  token,
  norm,
  stops,
  doc2Vec
])
empty_df = spark.createDataFrame([[""]]).toDF('text')
doc2vec_model = pipeline_doc2vec.fit(empty_df)

stopwords_en download started this may take some time.
Approximate size to download 2.9 KB
[OK!]
doc2vec_gigaword_wiki_300 download started this may take some time.
Approximate size to download 312.3 MB
[ | ]doc2vec_gigaword_wiki_300 download started this may take some time.
Approximate size to download 312.3 MB
[ — ]Download done! Loading the resource.
[ \ ]

[OK!]


In [102]:
with_doc2vec = doc2vec_model.transform(articles_abstracts).select("id",F.col("sentence_embeddings.embeddings")[0].alias("embeddings"))
with_doc2vec.cache()
# only 1m 18.5s

DataFrame[id: bigint, embeddings: array<float>]

In [103]:
with_doc2vec.head()

Row(id=0, embeddings=[0.02049514278769493, 0.0027027390897274017, -0.02206415683031082, -0.014081479050219059, 0.02675016038119793, -0.0025557605549693108, -0.03921186923980713, -0.022662989795207977, 0.01931287720799446, -0.028936661779880524, 0.015005095861852169, -0.017352813854813576, 0.014226953499019146, -0.0012490303488448262, -0.0038129265885800123, 0.003299228847026825, -0.012594044208526611, 0.00601203041151166, 0.017957666888833046, 0.024490540847182274, -0.04826386272907257, 0.025399409234523773, -0.0011097918031737208, 0.042576540261507034, -0.023143433034420013, 0.004230480175465345, -0.02783290483057499, -0.05379168316721916, -0.003942739684134722, 0.0272237379103899, -0.022858165204524994, 0.011684686876833439, 0.0025437287986278534, -0.004211811814457178, -0.030333364382386208, 0.0006534804706461728, 0.0027076248079538345, -0.010169739834964275, 0.014752971939742565, 0.03539455309510231, 0.019348373636603355, -0.009497113525867462, 0.02914026379585266, -0.0071002184413

In [104]:
query = """match (a:Article{id:event.id})
set a.embeddings=event.embeddings"""

with_doc2vec.write.format("org.neo4j.spark.DataSource")\
.option("url", "bolt://192.168.0.178:7687")\
.option("authentication.basic.username", os.environ["NEO4J_LOGIN"])\
.option("authentication.basic.password", os.environ["NEO4J_PASSWORD"])\
.mode("append")\
.option("query", query)\
.save()
# 1m 59s upload to database

In [105]:
with_stems = stemmer_model.transform(articles_abstracts).select("id","cleanedStem.result")#,F.col("normalized.result"))
with_stems.cache()
# 1m 9.8s

DataFrame[id: bigint, result: array<string>]

In [113]:
with_stems.select(F.size(F.col("result")).alias("size")).agg(F.sum("size")).first()

Row(sum(size)=16144851)

In [120]:
with_stems.head(1)

KeyboardInterrupt: 

22/06/21 11:18:03 ERROR OverwriteByExpressionExec: Data source write support org.neo4j.spark.writer.Neo4jBatchWriter@627342ad is aborting.
22/06/21 11:18:03 ERROR OverwriteByExpressionExec: Data source write support org.neo4j.spark.writer.Neo4jBatchWriter@627342ad aborted.


In [117]:
query = """match (a:Article{id:event.id})
with a,event
UNWIND event.result as stem
MERGE (s:Stem{word:stem})
with a,s
MERGE (a)-[rel:CONTAINS]->(s)
ON CREATE SET rel.weight = 1
ON MATCH SET rel.weight = rel.weight+1"""

with_stems.write.format("org.neo4j.spark.DataSource")\
.option("url", "bolt://192.168.0.178:7687")\
.option("authentication.basic.username", os.environ["NEO4J_LOGIN"])\
.option("authentication.basic.password", os.environ["NEO4J_PASSWORD"])\
.option("transaction.retries", 5)\
.option("transaction.retry.timeout", 5)\
.mode("Overwrite")\
.option("query", query)\
.save()


KeyboardInterrupt: 

In [57]:
from sparknlp.annotator import PerceptronModel,SentenceDetector,Tokenizer,Stemmer,Normalizer,StopWordsCleaner
from sparknlp.base import DocumentAssembler,Pipeline,LightPipeline

document_assembler = DocumentAssembler() \
  .setInputCol("text") \
  .setOutputCol("document")

sentence = SentenceDetector() \
    .setInputCols("document") \
    .setOutputCol("sentence")

tokenizer = Tokenizer() \
    .setInputCols(["sentence"]) \
    .setOutputCol("token")

stemmer = Stemmer() \
  .setInputCols(["token"]) \
  .setOutputCol("stem")

norm = Normalizer()\
  .setInputCols(["token"])\
  .setOutputCol("normalized")\
  .setLowercase(True)

stops = StopWordsCleaner.pretrained()\
  .setInputCols("normalized")\
  .setOutputCol("cleanedStem")


stem_pipeline = Pipeline(stages=[
  document_assembler,
  sentence,
  tokenizer,
  stemmer,
  norm,
  stops,
])

empty_df = spark.createDataFrame([[""]]).toDF('text')
stem_model = stem_pipeline.fit(empty_df)


stopwords_en download started this may take some time.
Approximate size to download 2.9 KB
[OK!]


In [61]:
with_stems = stem_model.transform(articles_abstracts).select("id","cleanedStem.result")#,F.col("normalized.result"))
with_stems.persist(StorageLevel.DISK_ONLY)

DataFrame[id: bigint, result: array<string>]

In [62]:
with_stems.head(1)

[Row(id=0, result=['evasion', 'attacks', 'machine', 'learning', 'test', 'time', 'securitysensitive', 'applications', 'success', 'machine', 'learning', 'depends', 'vetting', 'resistance', 'adversarial', 'data', 'pertinent', 'wellmotivated', 'attack', 'scenario', 'adversary', 'attempt', 'evade', 'deployed', 'system', 'test', 'time', 'carefully', 'manipulating', 'attack', 'samples', 'work', 'present', 'simple', 'effective', 'gradientbased', 'approach', 'exploited', 'systematically', 'assess', 'security', 'widelyused', 'classification', 'algorithms', 'evasion', 'attacks', 'recently', 'proposed', 'framework', 'security', 'evaluation', 'simulate', 'attack', 'scenarios', 'exhibit', 'risk', 'levels', 'classifier', 'increasing', 'attackers', 'knowledge', 'system', 'ability', 'manipulate', 'attack', 'samples', 'classifier', 'designer', 'picture', 'classifier', 'performance', 'evasion', 'attacks', 'perform', 'informed', 'model', 'selection', 'parameter', 'setting', 'evaluate', 'approach', 'releva

In [127]:

from pyspark.sql.types import ArrayType,StructType,StructField,StringType

def get_sliding_window(window_size:int):
    def window(arr:list):
        arrlen = len(arr)
        size = min(arrlen*(arrlen-1)//2,window_size*(window_size-1)//2) + max(0,(arrlen-window_size))*(window_size-1)
        result = [None] * size
        index = 0
        for i in range(size):
            for j in range(i+1,min(i+window_size,arrlen)):
                result[index] = (arr[i],arr[j]) if (arr[i]>arr[j]) else (arr[j],arr[i])
                index+=1
        return result
    return F.udf(window,ArrayType(StructType([StructField("first",StringType(),False),StructField("second",StringType(),False)])))

mySlidingPairs2 = get_sliding_window(7)
coocurence = with_stems.select(F.explode(mySlidingPairs2("result")).alias("result")).select("result.first","result.second").groupBy("first","second").count()

In [128]:
coocurence.describe

<bound method DataFrame.describe of DataFrame[first: string, second: string, count: bigint]>

In [129]:
coocurence.write.format("org.neo4j.spark.DataSource")\
.option("url", "bolt://192.168.0.178:7687")\
.option("authentication.basic.username", os.environ["NEO4J_LOGIN"])\
.option("authentication.basic.password", os.environ["NEO4J_PASSWORD"])\
.option("relationship", "COOCURES")\
.option("relationship.save.strategy", "keys")\
.option("relationship.properties", "count:weight")\
.option("relationship.source.labels", ":Stem")\
.option("relationship.source.save.mode", "Match")\
.option("relationship.source.node.keys", "first:word")\
.option("relationship.target.labels", ":Stem")\
.option("relationship.target.node.keys", "second:word")\
.option("relationship.target.save.mode", "Match")\
.option("transaction.retries", 5)\
.option("transaction.retry.timeout", 5)\
.mode("Append")\
.save()


In [79]:
distinct_stems = with_stems.select(F.explode("result").alias("result")).distinct()

In [81]:
query1 = """MERGE (:Stem{word:event.result})
"""

distinct_stems.write.format("org.neo4j.spark.DataSource")\
.option("url", "bolt://192.168.0.178:7687")\
.option("authentication.basic.username", os.environ["NEO4J_LOGIN"])\
.option("authentication.basic.password", os.environ["NEO4J_PASSWORD"])\
.option("transaction.retries", 5)\
.option("transaction.retry.timeout", 5)\
.mode("Overwrite")\
.option("query", query1)\
.save()

In [74]:
article_stem_relation = with_stems.select("id",F.explode("result").alias("result")).groupBy("id","result").count()

In [83]:
article_stem_relation.head(1)

[Row(id=10, result='markovian', count=1)]

In [88]:

# query2 = """match (a:Article{id:event.id}),(s:Stem{word:event.result})
# with a,s,event
# MERGE (a)-[rel:CONTAINS{weight:event.count}]->(s)"""

article_stem_relation.write.format("org.neo4j.spark.DataSource")\
.option("url", "bolt://192.168.0.178:7687")\
.option("authentication.basic.username", os.environ["NEO4J_LOGIN"])\
.option("authentication.basic.password", os.environ["NEO4J_PASSWORD"])\
.option("relationship", "CONTAINS")\
.option("relationship.save.strategy", "keys")\
.option("relationship.properties", "count:weight")\
.option("relationship.source.labels", ":Article")\
.option("relationship.source.save.mode", "Match")\
.option("relationship.source.node.keys", "id:id")\
.option("relationship.target.labels", ":Stem")\
.option("relationship.target.node.keys", "result:word")\
.option("relationship.target.save.mode", "Match")\
.option("transaction.retries", 5)\
.option("transaction.retry.timeout", 5)\
.mode("Append")\
.save()
